###  步骤
1.分维度把持续图分解出来
2。在每一个维度里面，定义一个阈值，把小于这个阈值的当成噪声筛掉。
3.bottleneck 的计算。函数里传入两个日期，计算两个日期持续图的bottleneck距离
4.拓扑特征（二维点的生存时间） 每个维度的平均生存时间和最大生存时间。


df
索引是日期：       拓扑标签1 2 3 4 5 6       涨幅（收益）

2024-0101           2.0 1.5                         





2024-09-12        2.0                              记数（>10） 







第二种： 因子和收益率相关性的




需要论文支撑（拓扑数据分析，应用于金融领域）

In [1]:
import pandas as pd
import numpy as np
import gudhi
import matplotlib.pyplot as plt
import pandas as pd
from gudhi import AlphaComplex
import gudhi as gd
import gudhi.alpha_complex as ac
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False  



In [2]:
def filter_table_by_date_and_items(df, date, items):
    '''
    输入：大表、日期、该日期下的hs300标的列表
    '''
    filtered_df = df[(df['name'] == date) & (df['Unnamed: 1'].isin(items))]
    return filtered_df

def get_non_empty_symbols_for_date(date, df):
    filtered_row = df[df['Date'] == date]
    if filtered_row.empty:
        return f"No data available for the specified date: {date}"
    row = filtered_row.iloc[0]
    non_empty_symbols = row.drop(labels=['Date']).dropna().index.tolist()
    return non_empty_symbols

def function(date,df_comp,df_x):
    '输入日期'

    filtered_row = df_comp[df_comp['Date'] == date]
    if filtered_row.empty:
        return f"No data available for the specified date: {date}"
    row = filtered_row.iloc[0]
    non_empty_symbols = row.drop(labels=['Date']).dropna().index.tolist()
    non_empty_symbols_numeric = list(map(int, non_empty_symbols))

    df = filter_table_by_date_and_items(df_x,date,non_empty_symbols_numeric)
    return df




def alpha_function(df):
    # 从 DataFrame 中选取数值类型的数据，随机选择5列
    numerical_data = df.select_dtypes(include=['float64', 'int64']).sample(n=5, axis=1).head(300)
    print(numerical_data)
    topology_points = numerical_data.to_numpy()
    print(topology_points)

    
    alpha_complex = gudhi.AlphaComplex(points=topology_points)
    simplex_tree = alpha_complex.create_simplex_tree()  
    diag = simplex_tree.persistence()
    

    # 绘制持久性图
    gudhi.plot_persistence_diagram(diag)
    plt.title('Topological Point Cloud - Persistence Diagram using Alpha Complex')
    plt.show()



def rips_output(date,df_comp,df_x):
    df = function(date,df_comp,df_x)
    alpha_function(df)
    return




stree.dimension(): 返回单纯形树的最大维度。例如，0表示点，1表示边，2表示三角形。  
stree.num_simplices(): 返回单纯形的总数，包括点、边和更高维的单纯形。  
stree.num_vertices(): 返回单纯形树中的顶点数，即 Alpha 复杂中点的数量。  

stree.get_filtration(): 返回单纯形树中每个单纯形及其对应的过滤值。过滤值代表在生成 Alpha 复杂时，每个单纯形的出现顺序或时间（半径的大小）。

In [3]:
#单独把每一维度的实现出来(传入一个参数，调取指定维度的图)

def sustainability(date,df_comp,df_x):
    df = function(date,df_comp,df_x)
    dimension = 1
    numerical_data = df.select_dtypes(include=['float64', 'int64']).sample(n=3, axis=1).head(300)#随机选n列特征绘制持续性图
    topology_points = numerical_data.to_numpy()
    ac = AlphaComplex(points=topology_points)
    stree = ac.create_simplex_tree()
    diag = stree.persistence()
    print('Alpha complex is of dimension ', stree.dimension(), ' - ',
    stree.num_simplices(), ' simplices - ', stree.num_vertices(), ' vertices.')

    fmt = '%s -> %.2f'
    for filtered_value in stree.get_filtration():
        print(fmt % tuple(filtered_value))


    filtered_diag = [pair for pair in diag if pair[0] == dimension]

    # 绘制持久性图
    gudhi.plot_persistence_diagram(diag)
    plt.title(f'拓扑点云 - {dimension} 维持久性图')
    plt.show()


In [4]:

def bottleneck_fun(date1,date2,df_comp,df_x):
    df1 = function(date1,df_comp,df_x)
    df2 = function(date2,df_comp,df_x)
    #后续再换
    df1 = df1.iloc[:,-5:-1]
    df2 = df2.iloc[:,-5:-1]
   
    points1 = df1.values
    points2 = df2.values


    alpha_complex1 = ac.AlphaComplex(points=points1)
    simplex_tree1 = alpha_complex1.create_simplex_tree()
    diag1 = simplex_tree1.persistence()

    alpha_complex2 = ac.AlphaComplex(points=points2)
    simplex_tree2 = alpha_complex2.create_simplex_tree()
    diag2 = simplex_tree2.persistence()


    diag1_np = np.array([[birth, death] for dim, (birth, death) in diag1])
    diag2_np = np.array([[birth, death] for dim, (birth, death) in diag2])

    # 计算 Bottleneck 距离
    bottleneck_distance = gd.bottleneck_distance(diag1_np, diag2_np)
    print(f"Bottleneck 距离: {bottleneck_distance}")
    return bottleneck_distance




: 

In [5]:
df_comp = pd.read_csv('comp_000300.csv')
df_quote = pd.read_csv('quote_000300.csv')
df_x = pd.read_csv('x.csv')

date1 = '2024-01-04'
date2 = '2024-01-05'
sustainability(date1,df_comp,df_x)
bottleneck_fun(date1,date2,df_comp,df_x)

C:\Windows\Temp\ipykernel_10596\3060910385.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_x = pd.read_csv('x.csv')
